<a href="https://colab.research.google.com/github/v0o0v/xUnit/blob/master/19%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 19장 테이블 차리기


ToDO List
* [X] 테스트 메서드 호출하기
* **먼저 SetUp 호출하기**
* 나중에 tearDown 호출하기
* 테스트 메서드가 실패하더라도 tearDown 호출하기
* 여러 개의 테스트 실행하기
* 수집된 결과를 출력하기 

---
테스트는 공통된 패턴이 있다

1. 준비(arrange) - 객체를 생성한다.
2. 행동(act) - 어떤 자극을 준다.
3. 확인(assert) - 결과를 검사한다.

1번은 2/3번과 다르게 동일한 경우가 많다.
이런 패턴으로 인해 테스트를 위해 객체를 얼마나 자주 생성해야 하는지에 대한 제약이 상충한다.
- 성능/격리 : 격리하기 위해 매 테스트마다 객체를 생성하면 성능이 떨어진다. 하지만 성능을 위해서 객체를 재사용하게 되면 테스트간에 커플링으로 인해 다른 문제가 발생한다.

일단, 객체 생성이 충분히 빠르다는 가정하에 모든 테스트마다 객체를 생성한다고 가정한다.

일단 테스트 코드 추가.

메타포 : 격리 / 플래그

In [0]:
class TestCaseTest(TestCase):
    def testRunning(self):
        test = WasRun("testMethod")
        assert(not test.wasRun)
        test.run()
        assert(test.wasRun)

    # 추가
    def testSetup(self):
        test = WasRun("testMethod") # 격리
        test.run()
        assert(test.wasSetup)

TestCaseTest("testRunning").run()
TestCaseTest("testSetup").run() #추가

당연히 실패가 뜬다.
성공시키기 위해 코드 추가.

In [0]:
class WasRun(TestCase):
  def __init__(self, name):
    self.wasRun = None
    TestCase.__init__(self,name)

  def testMethod(self):
    self.wasRun=1

  # 추가
  def setUp(self):
      self.wasSetup=1 # 플래그

wasRun의 테스트를 실행하는 곳은 TestCase이기 때문에 setUp을 호출하는 주체도 TestCase이다.

In [0]:
class TestCase:
  def __init__(self, name):
    self.name = name

  def run(self):
    self.setUp() # 추가
    method = getattr(self, self.name)
    method()

  # 추가
  def setUp(self):
    pass

다시 테스트.

In [0]:
TestCaseTest("testRunning").run()
TestCaseTest("testSetup").run() #추가

테스트 성공.

---

테스트 케이스 수행될 때는 setUp() -> testMethod() 으로 두 단계를 거친다.
따라서, 중복적인 요소를 제거할 수 있다.


메타포 : 중복제거

- wasRun 플래그를 setUp()에서 설정한다.

In [0]:
class WasRun(TestCase):
  def __init__(self, name):    
    TestCase.__init__(self,name)

  def testMethod(self):
    self.wasRun=1

  def setUp(self):
    self.wasRun = None #이동
    self.wasSetup=1

- test 전에 wasRun 플래그가 false인지 확인을 setUp() 테스트에서만 한다.

In [0]:
class TestCaseTest(TestCase):
    def testRunning(self):
        test = WasRun("testMethod")
        # assert(not test.wasRun) 삭제
        test.run()
        assert(test.wasRun)

    def testSetup(self):
        test = WasRun("testMethod")
        test.run()
        assert(test.wasSetup)

- WasRun 인스턴스를 setUp()에서 생성하고 testMethod()에서 같이 사용한다. 테스트마다 새로 생성하기 때문에 격리 문제는 발생하지 않는다.


In [0]:
class TestCaseTest(TestCase):

    #추가
    def setUp(self):
      self.test = WasRun("testMethod")

    def testRunning(self):        
        self.test.run()
        assert(self.test.wasRun)

    def testSetup(self):
        self.test.run()
        assert(self.test.wasSetup)

TestCaseTest("testRunning").run()
TestCaseTest("testSetup").run()

ToDO List
* [X] 테스트 메서드 호출하기
* [X] **먼저 SetUp 호출하기**
* 나중에 tearDown 호출하기
* 테스트 메서드가 실패하더라도 tearDown 호출하기
* 여러 개의 테스트 실행하기
* 수집된 결과를 출력하기 